In [1]:
import requests, bs4
import mysql.connector
from lxml import html
from urllib.request import Request,urlopen
from urllib.parse import urlencode,quote_plus
from bs4 import BeautifulSoup
from xml.etree.ElementTree import parse
from datetime import datetime
from ServiceKey import public_serviceKey, mySQL_Connect

mydb = mySQL_Connect()
# 2021.10.06 데이터 추출 완료 - 31140 데이터가 대부분 없음, InterfaceError: Python type ResultSet cannot be converted > updowncd 값 2에 의한 에러
cursor = mydb.cursor(prepared=True)
citycode = '35040'

query1 = ("select citycode from public_busctycodelist where citycode >= {}".format(citycode))

cursor.execute(query1)

location_list1 = []

for a in cursor:
    location_list1.append(a)

now = datetime.now()
nowDate = now.strftime('%Y%m%d')

b_cnt = 0
for z in location_list1:
    if b_cnt == 1:
        break
        
    location = z
    query2 = ("select distinct routeid from public_busroute_info where citycode = '{}'".format(location[0]))
    cursor.execute(query2)

    location_list2 = []

    for b in cursor:
        location_list2.append(b)

    cnt = 1
    for i in location_list2:
        routeId = i

        characters = "(',')"

        join_data = ''.join( x for x in routeId if x not in characters)
        print("조회날짜 : {} | location : {} | cnt : {} | start {} public_busroute_info".format(nowDate,location[0],cnt,join_data))

        url = 'http://openapi.tago.go.kr/openapi/service/BusRouteInfoInqireService/getRouteAcctoThrghSttnList'
        queryParams = '?' + urlencode({
            quote_plus('serviceKey') : public_serviceKey(),
            quote_plus('cityCode') : location[0],
            quote_plus('pageNo') : '1',
            quote_plus('numOfRows') : '10000',
            quote_plus('routeId') : join_data
        })

        response = requests.get(url + queryParams).text.encode('utf-8')
        xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')

        rows = xmlobj.findAll('item')
        resultCode = xmlobj.findAll('resultCode')
        if resultCode[0].text == '99':
            print("location {} | {} 번째부터 조회수 제한이 걸렸습니다.".format(location[0],join_data))
            mydb.close()
            b_cnt += 1
            break
        if rows == []:
            print("location {} | {} 데이터가 없습니다.".format(location[0],join_data))
            cnt += 1
            continue
        cnt += 1
        for j in rows:
            item = j
            lat = item.findAll('gpslati')                
            lng = item.findAll('gpslong')
            nodeid = item.findAll('nodeid')
            nodenm = item.findAll('nodenm')
            nodeno = item.findAll('nodeno')
            if nodeno == []:
                nodeno = None
            else:
                nodeno = nodeno[0].text
            nodeord = item.findAll('nodeord')
            routeid = item.findAll('routeid')  
            updowncd = item.findAll('updowncd')
            
            if updowncd == []:
                updowncd = None
            elif updowncd[0].text == '0':
                updowncd = '상행'
            elif updowncd[0].text == '1':
                updowncd = '하행'
            elif updowncd[0].text == '2':
                updowncd = '2'

            sql = "INSERT INTO public_busroute_detail (lat, lng, nodeid, nodenm, nodeno, nodeord, routeid, updowncd, citycode) VALUES (?,?,?,?,?,?,?,?,?)"
            val = (lat[0].text, lng[0].text, nodeid[0].text, nodenm[0].text, nodeno, nodeord[0].text, routeid[0].text, updowncd, location[0])

            cursor.execute(sql, val)
            mydb.commit()

조회날짜 : 20211006 | location : 35010 | cnt : 1 | start JUB305000997 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 2 | start JUB305001010 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 3 | start JUB305001065 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 4 | start JUB305001066 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 5 | start JUB305001067 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 6 | start JUB305001068 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 7 | start JUB305001076 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 8 | start JUB305001077 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 9 | start JUB305001086 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 10 | start JUB305001087 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 11 | start JUB305001088 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt 

조회날짜 : 20211006 | location : 35010 | cnt : 95 | start JUB305001432 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 96 | start JUB305001441 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 97 | start JUB305001442 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 98 | start JUB305001443 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 99 | start JUB305001466 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 100 | start JUB305001467 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 101 | start JUB305001471 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 102 | start JUB305001473 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 103 | start JUB305001475 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 104 | start JUB305001476 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 105 | start JUB305001477 public_busroute_info
조회날짜 : 20211006 | location

조회날짜 : 20211006 | location : 35010 | cnt : 190 | start JUB305001543 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 191 | start JUB305001544 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 192 | start JUB305001545 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 193 | start JUB305001546 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 194 | start JUB305001547 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 195 | start JUB305001548 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 196 | start JUB305001549 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 197 | start JUB305001552 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 198 | start JUB305001553 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 199 | start JUB305001554 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 200 | start JUB305300238 public_busroute_info
조회날짜 : 20211006 | loc

조회날짜 : 20211006 | location : 35010 | cnt : 284 | start JUB305001261 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 285 | start JUB305001262 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 286 | start JUB305001309 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 287 | start JUB305001310 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 288 | start JUB305001311 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 289 | start JUB305001312 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 290 | start JUB305001313 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 291 | start JUB305001314 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 292 | start JUB305001315 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 293 | start JUB305001317 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 294 | start JUB305001318 public_busroute_info
조회날짜 : 20211006 | loc

조회날짜 : 20211006 | location : 35010 | cnt : 377 | start JUB305001278 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 378 | start JUB305001389 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 379 | start JUB305001409 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 380 | start JUB305001127 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 381 | start JUB305001128 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 382 | start JUB305001129 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 383 | start JUB305001229 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 384 | start JUB305001230 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 385 | start JUB305001257 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 386 | start JUB305001258 public_busroute_info
조회날짜 : 20211006 | location : 35010 | cnt : 387 | start JUB305001265 public_busroute_info
조회날짜 : 20211006 | loc

...

조회날짜 : 20211006 | location : 35030 | cnt : 96 | start TSB308000560 public_busroute_info
조회날짜 : 20211006 | location : 35030 | cnt : 97 | start TSB308000621 public_busroute_info
조회날짜 : 20211006 | location : 35030 | cnt : 98 | start TSB308000623 public_busroute_info
조회날짜 : 20211006 | location : 35030 | cnt : 99 | start TSB308000628 public_busroute_info
조회날짜 : 20211006 | location : 35030 | cnt : 100 | start TSB308000638 public_busroute_info
조회날짜 : 20211006 | location : 35030 | cnt : 101 | start TSB308000639 public_busroute_info
조회날짜 : 20211006 | location : 35030 | cnt : 102 | start TSB308000644 public_busroute_info
조회날짜 : 20211006 | location : 35030 | cnt : 103 | start TSB308000648 public_busroute_info
조회날짜 : 20211006 | location : 35030 | cnt : 104 | start TSB308000673 public_busroute_info
조회날짜 : 20211006 | location : 35030 | cnt : 105 | start TSB308000711 public_busroute_info
조회날짜 : 20211006 | location : 35030 | cnt : 106 | start TSB308000712 public_busroute_info
조회날짜 : 20211006 | locatio

조회날짜 : 20211006 | location : 35040 | cnt : 2 | start TSB309000002 public_busroute_info
조회날짜 : 20211006 | location : 35040 | cnt : 3 | start TSB309000003 public_busroute_info
조회날짜 : 20211006 | location : 35040 | cnt : 4 | start TSB309000004 public_busroute_info
조회날짜 : 20211006 | location : 35040 | cnt : 5 | start TSB309000005 public_busroute_info
조회날짜 : 20211006 | location : 35040 | cnt : 6 | start TSB309000006 public_busroute_info
조회날짜 : 20211006 | location : 35040 | cnt : 7 | start TSB309000007 public_busroute_info
조회날짜 : 20211006 | location : 35040 | cnt : 8 | start TSB309000008 public_busroute_info
조회날짜 : 20211006 | location : 35040 | cnt : 9 | start TSB309000009 public_busroute_info
조회날짜 : 20211006 | location : 35040 | cnt : 10 | start TSB309000010 public_busroute_info
조회날짜 : 20211006 | location : 35040 | cnt : 11 | start TSB309000011 public_busroute_info
조회날짜 : 20211006 | location : 35040 | cnt : 12 | start TSB309000012 public_busroute_info
조회날짜 : 20211006 | location : 35040 | cnt

조회날짜 : 20211006 | location : 35040 | cnt : 96 | start TSB309000136 public_busroute_info
조회날짜 : 20211006 | location : 35040 | cnt : 97 | start TSB309000138 public_busroute_info
조회날짜 : 20211006 | location : 35040 | cnt : 98 | start TSB309000139 public_busroute_info
조회날짜 : 20211006 | location : 35040 | cnt : 99 | start TSB309000140 public_busroute_info
조회날짜 : 20211006 | location : 35040 | cnt : 100 | start TSB309000141 public_busroute_info
조회날짜 : 20211006 | location : 35040 | cnt : 101 | start TSB309000156 public_busroute_info
조회날짜 : 20211006 | location : 35040 | cnt : 102 | start TSB309000161 public_busroute_info
조회날짜 : 20211006 | location : 35040 | cnt : 103 | start TSB309000162 public_busroute_info
조회날짜 : 20211006 | location : 35040 | cnt : 104 | start TSB309000163 public_busroute_info
조회날짜 : 20211006 | location : 35040 | cnt : 105 | start TSB309000164 public_busroute_info
조회날짜 : 20211006 | location : 35040 | cnt : 106 | start TSB309000165 public_busroute_info
조회날짜 : 20211006 | locatio

조회날짜 : 20211006 | location : 35040 | cnt : 190 | start TSB309000291 public_busroute_info
조회날짜 : 20211006 | location : 35040 | cnt : 191 | start TSB309000292 public_busroute_info
조회날짜 : 20211006 | location : 35040 | cnt : 192 | start TSB309000293 public_busroute_info
조회날짜 : 20211006 | location : 35040 | cnt : 193 | start TSB309000294 public_busroute_info
조회날짜 : 20211006 | location : 35040 | cnt : 194 | start TSB309000295 public_busroute_info
조회날짜 : 20211006 | location : 35040 | cnt : 195 | start TSB309000296 public_busroute_info
조회날짜 : 20211006 | location : 35040 | cnt : 196 | start TSB309000297 public_busroute_info
조회날짜 : 20211006 | location : 35040 | cnt : 197 | start TSB309000298 public_busroute_info
조회날짜 : 20211006 | location : 35040 | cnt : 198 | start TSB309000299 public_busroute_info
조회날짜 : 20211006 | location : 35040 | cnt : 199 | start TSB309000300 public_busroute_info
조회날짜 : 20211006 | location : 35040 | cnt : 200 | start TSB309000301 public_busroute_info
조회날짜 : 20211006 | loc

조회날짜 : 20211006 | location : 35040 | cnt : 283 | start TSB309000452 public_busroute_info
조회날짜 : 20211006 | location : 35040 | cnt : 284 | start TSB309000453 public_busroute_info
조회날짜 : 20211006 | location : 35040 | cnt : 285 | start TSB309000454 public_busroute_info
조회날짜 : 20211006 | location : 35040 | cnt : 286 | start TSB309000456 public_busroute_info
조회날짜 : 20211006 | location : 35040 | cnt : 287 | start TSB309000458 public_busroute_info
조회날짜 : 20211006 | location : 35040 | cnt : 288 | start TSB309000460 public_busroute_info
조회날짜 : 20211006 | location : 35040 | cnt : 289 | start TSB309000461 public_busroute_info
조회날짜 : 20211006 | location : 35040 | cnt : 290 | start TSB309000463 public_busroute_info
조회날짜 : 20211006 | location : 35040 | cnt : 291 | start TSB309000464 public_busroute_info
조회날짜 : 20211006 | location : 35040 | cnt : 292 | start TSB309000468 public_busroute_info
조회날짜 : 20211006 | location : 35040 | cnt : 293 | start TSB309000469 public_busroute_info
조회날짜 : 20211006 | loc